In [ ]:
## 필요한 라이브러리 로딩
import pandas as pd
from datetime import datetime, timedelta
from pandas.api.types import CategoricalDtype
pd.set_option('mode.chained_assignment',  None)

## 1. covid19 원본 데이터셋 로딩
## covid19 데이터 로딩(파일을 다운로드 받은 경우)
df_covid19 = pd.read_csv("owid-covid-data.csv")

## covid19 데이터 로딩(온라인에서 바로 로딩할 경우)
## df_covid19 = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")
## 2. 전체 데이터셋 중 최근 100일간의 데이터를 필터링한 df_covid19_100 생성
## df_covid19['date']를 datetime으로 변환
df_covid19['date'] = pd.to_datetime(df_covid19['date'], format="%Y-%m-%d")

## 대륙 데이터와 최종 데이터로부터 100일 전 데이터 필터링
df_covid19_100 = df_covid19[(df_covid19['iso_code'].isin(['KOR', 'OWID_ASI', 'OWID_EUR', 
'OWID_OCE', 'OWID_NAM', 'OWID_SAM', 'OWID_AFR'])) 
& (df_covid19['date'] >= (max(df_covid19['date']) - timedelta(days = 100)))]

## 대륙명을 한글로 변환
df_covid19_100.loc[df_covid19_100['location'] == 'South Korea', "location"] = '한국'
df_covid19_100.loc[df_covid19_100['location'] == 'Asia', "location"] = '아시아'
df_covid19_100.loc[df_covid19_100['location'] == 'Europe', "location"] = '유럽'
df_covid19_100.loc[df_covid19_100['location'] == 'Oceania', "location"] = '오세아니아'
df_covid19_100.loc[df_covid19_100['location'] == 'North America', "location"] = '북미'
df_covid19_100.loc[df_covid19_100['location'] == 'South America', "location"] = '남미'
df_covid19_100.loc[df_covid19_100['location'] == 'Africa', "location"] = '아프리카'

## 이산형 변수 설정
ord = CategoricalDtype(categories = ['한국', '아시아', '유럽', '북미', '남미', 
'아프리카','오세아니아'], ordered = True)

df_covid19_100['location'] = df_covid19_100['location'].astype(ord)

## date로 정렬
df_covid19_100 = df_covid19_100.sort_values(by = 'date')

## 3. df_covid19_100을 한국과 각 대륙별 열로 배치한 넓은 형태의 데이터프레임으로 변환
df_covid19_100_wide = df_covid19_100.loc[:,['date', 'location', 'new_cases', 
'people_fully_vaccinated_per_hundred']].rename(columns={'new_cases':'확진자', 
'people_fully_vaccinated_per_hundred':'백신접종완료자'})

df_covid19_100_wide = df_covid19_100_wide.pivot(index='date', columns='location', 
values=['확진자', '백신접종완료자']).sort_values(by = 'date')

df_covid19_100_wide.columns = ['확진자_한국', '확진자_아시아', '확진자_유럽', '확진자_북미',  
'확진자_남미', '확진자_아프리카', '확진자_오세아니아', '백신접종완료자_한국', 
'백신접종완료자_아시아', '백신접종완료자_유럽', '백신접종완료자_북미', 
'백신접종완료자_남미', '백신접종완료자_아프리카', '백신접종완료자_오세아니아']

## 4. covid19 데이터를 국가별로 요약한 df_covid19_stat 생성
df_covid19_stat = df_covid19.groupby(['iso_code', 'continent', 'location'], dropna=True).agg(
    인구수 = ('population', 'max'), 
    전체사망자수 = ('new_deaths', 'sum'), 
    백신접종자완료자수 = ('people_fully_vaccinated', 'max'), 
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred', 'max'), 
    인구백명당부스터접종자수 = ('total_boosters_per_hundred', 'max')).reset_index()

df_covid19_stat['십만명당사망자수'] = round(df_covid19_stat['전체사망자수'] / 
                                    df_covid19_stat['인구수'] * 100000, 5)

df_covid19_stat['백신접종완료율'] = df_covid19_stat['백신접종자완료자수'] / \
df_covid19_stat['인구수']

## 여백 설정을 위한 변수 설정
margins_P = {'t' : 50, 'b' : 25, 'l' : 25, 'r' : 25}

In [ ]:
## 파이썬 코드
## 대학 학과 취업률 데이터 로딩
df_취업률 = pd.read_excel("2021_employment_rate.xlsx", \
                       ## '학과별' 시트의 데이터를 불러오는데,
                       sheet_name = '학과별', \
                       ## 앞의 13행을 제외하고 \
                       skiprows=(13), \
                       ## 첫 번째 행은 열 이름으로 설정
                       header = 0)

## df_취업률에서 첫 번째부터 9번째까지의 열과 '계'로 끝나는 열을 선택하여 다시 df_취업률에 저장
df_취업률 = pd.concat([df_취업률.iloc[:, 0:8],
                    df_취업률.loc[:, df_취업률.columns.str.endswith('계')], \
                    df_취업률.loc[:, '입대자']], \
                    axis = 1)

## df_취업률에서 졸업자가 500명 이하인 학과 중 25% 샘플링
df_취업률_500 = df_취업률.loc[(df_취업률['졸업자_계'] < 500)]
df_취업률_500 = df_취업률_500.iloc[range(0, len(df_취업률_500.index) , 4)]
df_취업률_500 = df_취업률_500.rename(columns = {'졸업자_계':'졸업자수', '취업률_계': 
'취업률', '취업자_합계_계':'취업자수'})

In [ ]:
# Scatter 

import plotly.graph_objects as go

fig = go.Figure()

colors = {'의약계열':0, '인문계열':1, '사회계열':2, '교육계열':3, '공학계열':4, '자연계열':5, '예체능계열':6}

# 색상 설정을 위해 '대계열'로 그룹화해서 for 루프 사용
for cat, group in df_취업률_500.groupby('대계열'):
    # scatter trace 생성
    fig.add_trace(go.Scatter(
        mode = 'markers',
        x = group['졸업자수'], y = group['취업자수'],
        # 트레이스 이름 설정
        name = cat,
        # 마커 색상 설정
        marker = dict(color = colors[cat]),
        showlegend = True
    ))
# layout 속성 설정
fig.update_layout(
    # 제목 설정
    title = dict(text = '<b>졸업자 대비 취업자수</b>', x = 0.5, font = dict(color = 'white')),
    margin = margins_P, # 여백 설정
    paper_bgcolor = 'black', plot_bgcolor = 'black', # 배경색 설정
    # x축 설정
    xaxis = dict(color = 'white', ticksuffix = '명', showgrid = False),
    # y축 설정
    yaxis = dict(color = 'white', gridcolor = 'gray', ticksuffix = '명', dtick = 100),
    legend = dict(font = dict(color = 'white'))) # color scale 설정

fig.show()

In [ ]:
# plotly.express로 구현하면

import plotly.express as px

# px.scatter()로 scatter 트레이스 생성
fig = px.scatter(df_취업률_500, x = '졸업자수', y = '취업자수', color = '대계열')

# layout 설정
fig.update_layout(title = dict(text = '<b>졸업자 대비 취업자수</b>', 
                       x = 0.5, font = dict(color = 'white')),
                  margin = dict(t = 50, b = 25, l = 25, r = 25),
                  paper_bgcolor = 'black', plot_bgcolor = 'black',
                  xaxis = dict(color = 'white', ticksuffix = '명', showgrid = False),
                  yaxis = dict(color = 'white', gridcolor = 'gray',
                              ticksuffix = '명', dtick = 100),
                  legend = dict(font = dict(color = 'white')))

fig.show()

In [ ]:
import numpy as np 
import statsmodels.api as sm # to build a LOWESS model
from sklearn.linear_model import LinearRegression

# 선형회귀 모델
linear_regr = LinearRegression()
X = df_취업률_500['졸업자수'].values.reshape(-1, 1) # 독립변수(넘파이의 배열)
Y = df_취업률_500['취업자수'].values # 종속변수(넘파이의 배열)

linear_regr.fit(X, Y)# 선형 모델의 가중치를 학습
linear_fit = linear_regr.predict(X)

# Loess model
lowess_fit = sm.nonparametric.lowess(df_취업률_500['취업자수'], df_취업률_500['졸업자수'])

fig = go.Figure()

fig.add_trace(go.Scatter(
    mode = 'markers',
    x = df_취업률_500['졸업자수'], y = df_취업률_500['취업자수'],
    showlegend = False))

# 선형회귀 추세선 추가
fig.add_trace(go.Scatter(
    mode = 'lines',
    x = df_취업률_500['졸업자수'], y = linear_fit,
    name = '선형추세선',
    line = dict(dash='dot')))

# 국소회귀 추세선 추가
fig.add_trace(go.Scatter(
    mode = 'lines',
    x = lowess_fit[:,0], y = lowess_fit[:,1],
    name = 'lowess'))

fig.show()

In [ ]:
# plotly.exrpess의 trendline 매개변수로 간단한 추세선을 만들 수 있음
# ols(ordinary least squares regression line), lowess(국소선형회귀선), rolling(이동평균선), expanding(확장평균선), ewm(지수가중평균)

# 선형회귀 추세선
fig = px.scatter(df_취업률_500, x = '졸업자수', y = '취업자수', color = '대계열', trendline = 'ols')

fig.show()

# 국소회귀 추세선
fig = px.scatter(df_취업률_500, x = '졸업자수', y = '취업자수', color = '대계열', trendline = 'lowess')

fig.show()

In [ ]:
# 버블 차트

fig = go.Figure()

fig.add_trace(go.Scatter(
    mode = 'markers',
    x = df_covid19_stat['백신접종완료율'],
    y = df_covid19_stat['인구백명당부스터접종자수'],
    # marker의 사이즈를 사용해 버블차트 구현
    marker = dict(size = df_covid19_stat['십만명당사망자수'], opacity = 0.5,
                 sizemode = 'area')
))

In [ ]:
# 히스토그램

fig = go.Figure()

# historgram trace로 x축을 취업률로 매핑, name을 취업률로 설정
fig.add_trace(go.Histogram(x = df_취업률_500['취업률'], name = '취업률',
                           # xbins 속성설정
                           xbins = dict(start = 0, end = 100, size = 2.5)))

# 제목과 여백 설정

fig.update_layout(title = dict(text = '취업률 histogram', x = 0.5))

fig.show()

In [ ]:
fig = go.Figure()

for cat, group in df_취업률_500.groupby('과정구분'):
    fig.add_trace(go.Histogram(
        x = group['취업률'], name = cat,
        xbins = dict(size = 10), opacity = 0.4))
    
# histogram barmode를 'overlay'로 설정
fig.update_layout(barmode='overlay', title = dict(text = '취업률 Histogram', x = 0.5))

fig.show()

In [ ]:
# 누적 히스토그램

fig = go.Figure()

fig.add_trace(go.Histogram(x = df_취업률_500['취업률'], name = '취업률',
                          xbins = dict(start = 0, end = 100, size = 2.5),
                          # 누적 히스토그램 설정
                          cumulative = dict(enabled = True)))

fig.update_layout(title = dict(text = '취업률 Histogram', x = 0.5))

fig.show()

In [ ]:
# 히스토그램 함수 사용
# count: 사례수를 표시
# sum: bin에 속한 값의 합을 표시
# avg: 평균
# min, max: 최소, 최대
# count를 제외하고는 각각의 함수에 적용되어야 하는 변수를 y 속성에 설정해주어야 함

###
fig = go.Figure()

fig.add_trace(go.Histogram(x = df_취업률_500['대계열'],
                          # 히스토그램 막대 함수를 'count'로 설정
                          histfunc = 'count', showlegend = False))
fig.update_layout(title = dict(text = '취업률 Histogram', x = 0.5))

fig.show()

###
fig = go.Figure()

fig.add_trace(go.Histogram(x = df_취업률_500['대계열'], y = df_취업률_500['취업률'],
                          # 히스토그램 막대 함수를 'sum'으로 설정
                          histfunc = 'sum', showlegend = False))
fig.update_layout(title = dict(text = '취업률 Histogram', x = 0.5))

fig.show()

###
fig = go.Figure()

fig.add_trace(go.Histogram(x = df_취업률_500['대계열'], y = df_취업률_500['취업률'],
                          # 히스토그램 막대 함수를 'average'으로 설정
                          histfunc = 'avg', showlegend = False))
fig.update_layout(title = dict(text = '취업률 Histogram', x = 0.5))

fig.show()

###
fig = go.Figure()

fig.add_trace(go.Histogram(x = df_취업률_500['대계열'], y = df_취업률_500['취업률'],
                          # 히스토그램 막대 함수를 'max'으로 설정
                          histfunc = 'max', showlegend = False))
fig.update_layout(title = dict(text = '취업률 Histogram', x = 0.5))

fig.show()

In [ ]:
# 박스플롯

fig = go.Figure()

fig.add_trace(go.Box(
    x = df_취업률['대계열'], y = df_취업률['취업률_계']))

fig.update_layout(title = dict(text = '대학 계열별 취업률 분포', x = 0.5))

fig.show()

In [ ]:
# 평균, 표준편차가 포함된 박스 플롯

fig = go.Figure()

fig.add_trace(go.Box(
    x = df_취업률['대계열'], y = df_취업률['취업률_계'],
    # boxmean과 notched(움푹들어가는 설정) 설정
    boxmean = 'sd', notched = True))

fig.update_layout(title = dict(text = '대학 계열별 취업률 분포', x = 0.5))

fig.show()

In [ ]:
# 그룹 박스 플롯

fig = go.Figure()

# 과정별 box 트레이스 생성
fig.add_trace(go.Box(
    x = df_취업률.loc[df_취업률['과정구분'] == '전문대학과정', '대계열'],
    y = df_취업률['취업률_계'], name = '전문대학과정'))

fig.add_trace(go.Box(
    x = df_취업률.loc[df_취업률['과정구분'] == '대학과정', '대계열'],
    y = df_취업률['취업률_계'], name = '대학과정'))

fig.add_trace(go.Box(
    x = df_취업률.loc[df_취업률['과정구분'] == '대학원과정', '대계열'],
    y = df_취업률['취업률_계'], name = '대학원과정'))

# boxmode를 group으로 설정
fig.update_layout(boxmode = 'group',
                 title = dict(text = '학위과정별 취업률 분포', x = 0.5, xanchor = 'center'))

In [ ]:
# 지터 박스 플롯
# boxpoints = 전체데이터 all, 이상치만 표시 outliers, 극단적인 이상치 표시 suspectedoutliers, 표시 안하는 False
# jitter = 0~1 데이터가 흩뿌려지는 너비
# pointpos = -2~2 데이터 점이 표시되는 위치를 설정하는데 박스의 중심에서부터 거리를 설정

fig = go.Figure()

# 대륙별 확진자 box 트레이스 생성
fig.add_trace(go.Box(
    y = df_covid19_100_wide['확진자_한국'], name = '한국',
    # boxpoints, jitter, pointpos 설정
    boxpoints = 'all', jitter = 0.3, pointpos = -1.8))

fig.add_trace(go.Box(
    y = df_covid19_100_wide['확진자_아시아'], name = '아시아',
    boxpoints = 'all', jitter = 0.3, pointpos = -1.8))

fig.add_trace(go.Box(
    y = df_covid19_100_wide['확진자_유럽'], name = '유럽',
    boxpoints = 'all', jitter = 0.3, pointpos = -1.8))

fig.add_trace(go.Box(
    y = df_covid19_100_wide['확진자_북미'], name = '북미',
    boxpoints = 'all', jitter = 0.3, pointpos = -1.8))

fig.add_trace(go.Box(
    y = df_covid19_100_wide['확진자_남미'], name = '남미',
    boxpoints = 'all', jitter = 0.3, pointpos = -1.8))

fig.add_trace(go.Box(
    y = df_covid19_100_wide['확진자_아프리카'], name = '아프리카',
    boxpoints = 'all', jitter = 0.3, pointpos = -1.8))

fig.add_trace(go.Box(
    y = df_covid19_100_wide['확진자_오세아니아'], name = '오세아니아',
    boxpoints = 'all', jitter = 0.3, pointpos = -1.8))

fig.update_layout(title = dict(text = '한국 및 대륙별 일별 확진자 분포', x = 0.5),
                 xaxis = dict(title = '대륙명'),
                 yaxis = dict(title = '확진자수(명)'),
                 margin = margins_P,
                 paper_bgcolor = 'lightgray', plot_bgcolor = 'lightgray')

fig.show()

In [ ]:
# violin 플롯
# 실제 상한값과 하한값보다 길게 표시되는 단점이 있음

fig = go.Figure()

fig.add_trace(go.Violin(
    x = df_취업률['대계열'], y = df_취업률['취업률_계']))

fig.update_layout(title = dict(text = '대학 계열별 취업률 분포', x = 0.5))
fig.show()

In [ ]:
# 박스 플롯과 평균선이 포함된 바이올린 플롯

fig = go.Figure()

fig.add_trace(go.Violin(
    x = df_취업률['대계열'], y = df_취업률['취업률_계'],
    # 바이올린 내부 박스 표시
    box = dict(visible = True),
    # 평균선 표시
    meanline = dict(visible = True)))

fig.update_layout(title = dict(text = '대학 계열별 취업률 분포', x = 0.5))

fig.show()

In [ ]:
# 분리된 바이올린 플롯
# side 속성을 활용하여 두개의 그룹을 비교할 수 있음

fig = go.Figure()

# 전문대학과정 violin 트레이스 추가
fig.add_trace(go.Violin(
    x = df_취업률.loc[df_취업률['과정구분'] == '전문대학과정', '대계열'],
    y = df_취업률['취업률_계'], name = '전문대학',
    # side, box, meanline 속성의 설정
    side = 'negative', box = dict(visible = True, width = 0.5), # negative = 왼쪽, box = boxplot
    meanline = dict(visible = True, width = 1)))

# 대학과정 violin 트레이스 추가
fig.add_trace(go.Violin(
    x = df_취업률.loc[df_취업률['과정구분'] == '대학과정', '대계열'],
    y = df_취업률['취업률_계'], name = '대학',
    side = 'positive', box = dict(visible = True, width = 0.5),
    meanline = dict(visible = True, width = 1)))

fig.update_layout(title = dict(text = '대학 계열별 취업률 분포', x = 0.5),
                  margin = margins_P)

In [ ]:
# CHAPTER 8 비교와 구성의 시각화
# 수직막대그래프

df_취업률_대계열평균 = df_취업률.groupby('대계열').agg(취업률 = ('취업률_계', 'mean'))

fig = go.Figure()

# bar 트레이스 추가
fig.add_trace(go.Bar(
    x = df_취업률_대계열평균.index, y = df_취업률_대계열평균['취업률'],
    # text 설정
    text = df_취업률_대계열평균['취업률'],
    # textposition, texttemplate 설정
    textposition = 'inside', texttemplate = '%{y:.1f}'))

fig.update_layout(title = dict(text = '계열별 취업률 평균', x = 0.5),
                  yaxis = dict(ticksuffix = '%'))

fig.show()

In [ ]:
# 정렬 막대그래프

# 인구수가 천만명 이상의 국가중에 인구백명당접종완료율 top10 필터링
vaccine_top10 = df_covid19_stat.loc[df_covid19_stat['인구수'] > 10000000].sort_values\
(by = ['인구백명당백신접종완료율'], ascending=False).head(10).reset_index()

fig = go.Figure()

for continent, group in vaccine_top10.groupby('continent'):
    fig.add_trace(go.Bar(
        x = group['location'], y = group['인구백명당백신접종완료율'],
        name = continent,
        text = group['인구백명당백신접종완료율'], textposition = 'outside', texttemplate = '%{text}%',
        textfont = dict(color = 'black')))
    
fig.update_layout(title = dict(text = '완전 백신 접종률 상위 top 10 국가', x = 0.5),
                 xaxis = dict(title = '국가명', categoryorder = 'total descending'),
                 yaxis = dict(title = '백신접종완료율', ticksuffix = '%'),
                 margin = margins_P)

fig.show()

In [ ]:
# 수평 막대그래프
# 각 대륙별로 그룹화된 백신 접종률 top5를 비교 -> 인구수 설정으로 오세아니아 = 1개국만 나오기 때문에 총 26개국
# 26개국을 수직 막대그래프로 표현하기에는 좌우폭이 좁기때문에 수평 막대그래프를 채용

# 대륙별 백신접종완료율 top5 필터링
vaccine_top5_by_continent = df_covid19_stat.loc[df_covid19_stat['인구수']>10000000].\
sort_values(by=['continent', '인구백명당백신접종완료율'], ascending=False).\
groupby('continent').head(5).reset_index()

fig = go.Figure()

for continent, group in vaccine_top5_by_continent.groupby('continent'):
    fig.add_trace(go.Bar(
        y = group['location'], x = group['인구백명당백신접종완료율'],
        name = continent,
        text = group['인구백명당백신접종완료율'], textposition = 'outside',
        texttemplate = '%{text}%',
        textfont = dict(color = 'black'), orientation = 'h'))
    
fig.update_layout(title = dict(text = '대륙별 완전 백신 접종률 상위 top 5 국가', x = 0.5),
                 xaxis = dict(title = '백신접종완료율',
                             ticksuffix = '%', range = (0, 105)),
                 yaxis = dict(title = '', autorange = 'reversed'),
                 margin = margins_P, height = 800)

fig.show()

In [ ]:
# 그룹 막대그래프

# 계열별 취업률을 넓은 데이터 형태로 전처리
취업률_by_계열 = df_취업률.groupby(['과정구분', '대계열']).\
agg(취업률 = ('취업률_계', 'mean')).reset_index().\
pivot(index = '대계열', columns = '과정구분', values = '취업률')

fig = go.Figure()

# 과정별로 bar 트레이스 추가
fig.add_trace(go.Bar(
    x = 취업률_by_계열.index,
    y = 취업률_by_계열['전문대학과정'],
    name = '전문대학과정'))

fig.add_trace(go.Bar(
    x = 취업률_by_계열.index,
    y = 취업률_by_계열['대학과정'],
    name = '대학과정'))

fig.add_trace(go.Bar(
    x = 취업률_by_계열.index,
    y = 취업률_by_계열['대학원과정'],
    name = '대학원과정'))

# barmode, bargroupgap 설정
fig.update_layout(barmode = 'group', bargroupgap = 0.2,
                 title = '계열별 교육과정별 취업률 평균',
                 title_x = 0.5, margin = margins_P)
fig.show()

In [ ]:
# 이중 축 막대 그래프
# 하나 이상의 트레이스를 사용하는 경우에는 보통 X, Y축 중 한 축의 스케일은 공유하지만 나머지 한 축은 각각의 데이터에 대한 스케일을 가질 수 있다.
# 이를 x2, y2를 사용하여 배정

fig = go.Figure()

# bar 트레이스 추가, 색상 설정을 위해 continent로 그룹화하고 for 루프 실행

for 대륙, group in vaccine_top10.groupby('continent'):
    fig.add_trace(go.Bar(
        x = group['location'], y = group['인구백명당백신접종완료율'],
        name = 대륙,
        text = group['인구백명당백신접종완료율'], textposition = 'outside',
        texttemplate = '%{text}%',
        textfont = dict(color = 'black')))
    
# markers+text 모드인 scatter 트레이스 생성
fig.add_trace(go.Scatter(mode = 'markers+text',
                        # yaxis를 'y2'로 설정
                        name = '10만명당 사망자수', yaxis = 'y2',
                        x = vaccine_top10['location'],
                        y = vaccine_top10['십만명당사망자수'],
                        text = round(vaccine_top10['십만명당사망자수'], 1),
                        textposition = 'top center'))

fig.update_layout(
    title = dict(text = '완전 백신 접종률 상위 top 10 국가와 십만명당 사망자수',
                x = 0.5),
    xaxis = dict(title = '국가명', categoryorder = 'total descending'),
    yaxis = dict(title = '백신접종완료율', ticksuffix = '%'),
    # y2 축 설정
    yaxis2 = dict(title = '인구 10만명당 사망자수', side = 'right',
                 overlaying = 'y', range = (0, 300), ticksuffix = '명'),
    margin = dict(r = 100, t = 50), legend = dict(x=1.1)) # y축 추가로 범례와 가까워져서 이동
    
fig.show()

In [ ]:
fig = go.Figure()

# bar 트레이스 추가, 색상 설정을 위한 for루프
for continent, group in vaccine_top5_by_continent.groupby('continent'):
    fig.add_trace(go.Bar(
        y = group['location'], x = group['인구백명당백신접종완료율'],
        name = continent,
        text = group['인구백명당백신접종완료율'], textposition = 'outside',
        texttemplate = '%{text}%',
        textfont = dict(color = 'black'), orientation = 'h'))
    
# markers + text 모드인 scatter 트레이스 생성
fig.add_trace(
    go.Scatter(mode = 'markers+text',
              # xaxis를 'x2'로 설정
              xaxis = 'x2', name = '사망자수',
              x = vaccine_top5_by_continent['십만명당사망자수'],
              y = vaccine_top5_by_continent['location'],
              text = round(vaccine_top5_by_continent['십만명당사망자수'], 1),
              textposition = 'middle right'))

fig.update_layout(title = dict(text = '대륙별 완전 백신 접종률 상위 top5 국가',
                              x = 0.5),
                 xaxis = dict(title = '백신접종완료율', ticksuffix = '%', range = (0, 105)),
                 # xaxis2 축의 설정
                 xaxis2 = dict(title = dict(text = '인구10만명당 사망자수', standoff = 1),
                              side = 'top', overlaying = 'x',
                              range = (0, 700), ticksuffix = '명'),
                 yaxis = dict(title = '', autorange = 'reversed'),
                 margin = dict(r = 100, t = 80), height = 800)

fig.show()

In [ ]:
## 비율 막대그래프를 위한 데이터 전처리
df_covid19_stat_tmp = df_covid19.groupby(['iso_code', 'continent', 'location'],
dropna=False).agg(
    인구수 = ('population', 'max'),
    전체사망자수 = ('new_deaths', 'sum'),
    백신접종자완료자수 = ('people_fully_vaccinated', 'max'),
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred', 'max'),
    인구백명당부스터접종자수 = ('total_boosters_per_hundred', 'max')
).reset_index()

df_covid19_stat_tmp = df_covid19_stat_tmp.loc[df_covid19_stat_tmp['iso_code'].isin(
    ['OWID_HIC', 'OWID_LIC', 'OWID_LMC', 'OWID_UMC']), 
    ['location', '전체사망자수', '백신접종자완료자수', 
     '인구백명당백신접종완료율']].transpose()

df_covid19_stat_tmp = df_covid19_stat_tmp.rename(columns=df_covid19_stat_tmp.iloc[0]).\
drop(df_covid19_stat_tmp.index[0])

df_covid19_stat_tmp['sum'] = df_covid19_stat_tmp.sum(axis = 1)

df_covid19_stat_tmp['High income'] = \
df_covid19_stat_tmp['High income'] / df_covid19_stat_tmp['sum']

df_covid19_stat_tmp['Low income'] = \
df_covid19_stat_tmp['Low income'] / df_covid19_stat_tmp['sum']

df_covid19_stat_tmp['Lower middle income'] = \
df_covid19_stat_tmp['Lower middle income'] / df_covid19_stat_tmp['sum']

df_covid19_stat_tmp['Upper middle income'] = \
df_covid19_stat_tmp['Upper middle income'] / df_covid19_stat_tmp['sum']

fig = go.Figure()

## 'High income'을 위한 bar 트레이스 추가
fig.add_trace(go.Bar(
    x = df_covid19_stat_tmp['High income'],
    y = df_covid19_stat_tmp.index, orientation = 'h', name = 'High income',
    marker = dict(line = dict(color = 'white', width = 2))))

## 'Upper middle income'을 위한 bar 트레이스 추가
fig.add_trace(go.Bar(
    x = df_covid19_stat_tmp['Upper middle income'],
    y = df_covid19_stat_tmp.index, orientation = 'h', name = 'Upper middle income',
    marker = dict(line = dict(color = 'white', width = 2))))

## 'Lower middle income'을 위한 bar 트레이스 추가
fig.add_trace(go.Bar(
    x = df_covid19_stat_tmp['Lower middle income'],
    y = df_covid19_stat_tmp.index, orientation = 'h', name = 'Lower middle income',
    marker = dict(line = dict(color = 'white', width = 2))))

## 'Low income'을 위한 bar 트레이스 추가
fig.add_trace(go.Bar(
    x = df_covid19_stat_tmp['Low income'],
    y = df_covid19_stat_tmp.index, orientation = 'h', name = 'Low income',
    marker = dict(line = dict(color = 'white', width = 2))))

for index, row in df_covid19_stat_tmp.iterrows():
## 'High income' 값 표시를 위한 주석 레이어 추가
    fig.add_annotation(
    xref = 'x', yref = 'y',
    x = row['High income'] / 2, y = index,
    text = str(round(row['High income'] * 100, 1)) + '%',
    showarrow = False, font = dict(color = 'white'))
    
    ## 'Upper middle income' 값 표시를 위한 주석 레이어 추가
    fig.add_annotation(
    xref = 'x', yref = 'y',
    x = (row['Upper middle income'] / 2) + 
        row['High income'], y = index, 
    text = str(round(row['Upper middle income'] * 100, 1)) + '%',
    showarrow = False, font = dict(color = 'white'))
    
    ## 'Lower middle income' 값 표시를 위한 주석 레이어 추가
    fig.add_annotation(
    xref = 'x', yref = 'y',
    x = (row['Lower middle income'] / 2) + row['High income'] + 
        row['Upper middle income'], y = index,
    text = str(round(row['Lower middle income'] * 100, 1)) + '%',
    showarrow = False, font = dict(color = 'white'))
    
    ## 'Low income' 값 표시를 위한 주석 레이어 추가
    fig.add_annotation(
    xref = 'x', yref = 'y',
    x = (row['Low income'] / 2) + row['High income'] + row['Upper middle income'] +
        row['Lower middle income'], y = index,
    text = str(round(row['Low income'] * 100, 1)) + '%',
    showarrow = False, font = dict(color = 'white'))

fig.update_layout(barmode = 'stack',
    title = dict(text = '국가 소득 구간별 코로나19 현황', x = 0.5),
    xaxis = dict(title = '', tickformat = '.0%'),
    yaxis = dict(title = ''),
    legend = dict(orientation = 'h', traceorder = 'normal'),
    margin = margins_P)

fig.show()

In [ ]:
# 롤리팝 그래프

df_lolipop = df_covid19_stat.reset_index()
df_lolipop = df_lolipop.loc[(df_lolipop['인구수'] > 5000000) & 
            (df_lolipop['continent'] == 'Asia') & 
            (df_lolipop['인구백명당백신접종완료율'].isna() == False)].\
sort_values(by = '인구백명당백신접종완료율', ascending=False)

fig = go.Figure()

## 라인 세그컨트 레이어를 추가
for index, row in df_lolipop.iterrows():
    fig.add_shape(type='line', xref='x', yref='y',
                x0 = row['location'], y0=0, x1=row['location'], 
                y1=row['인구백명당백신접종완료율'],
        line=dict(color='RoyalBlue',width=3))

## markers 모드인 scatter 트레이스 추가
fig.add_trace(go.Scatter(
    mode = 'markers', name = '접종완료율',
    x = df_lolipop['location'], 
    y = df_lolipop['인구백명당백신접종완료율'],
    marker = dict(color = 'darkblue', symbol = 0, size = 8) ))
fig.add_trace(go.Scatter(
    mode = 'markers', name = '사망자수',
    x = df_lolipop['location'], y = df_lolipop['십만명당사망자수'], 
    yaxis = "y2", 
    marker = dict(color = 'black', symbol = 100, size = 8)))

fig.update_layout(
    title = dict(text = '아시아 국가의 백신접종률', x = 0.5),
    xaxis = dict(title = ''),
    yaxis = dict(title = '백신접종완료율', range = (0, 105),
    ticksuffix = '%'),
    ## 두 번째 Y축의 설정
    yaxis2 = dict(title = dict(text = '인구10만명당 사망자수', 
                               standoff = 10),
    side = "right", overlaying = "y",
    range = (0, 200), ticksuffix = '명'),
    margin = margins_P, showlegend = True,
    legend = dict(orientation = 'h', y = -0.5, x = 0.5, 
                xanchor = 'center'))

fig.show()

In [ ]:
# 레이더 차트
# theta = 각도로 표현되는 축을 설정
# r = 반지름
# fill = 단색으로 채울 영역
# - toself = 트레이스의 시작점과 끝점을 연결해 닫힌 모양, 설정된 색으로 채움
# - tonext = 시작점과 끝점을 연결해 닫힌 모양으로 만들고 내부를 채우는데, 완전히 겹쳐지는 트레이스의 색을 설정된 색으로 사용

## 레이더 차트를 위한 데이터 전처리
df_radar_vaccine = df_covid19.groupby(['iso_code', 'continent', 'location'], dropna=False).agg(
    인구수 = ('population', 'max'),
    전체사망자수 = ('new_deaths', 'sum'),
    백신접종자완료자수 = ('people_fully_vaccinated', 'max'),
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred', 'max'),
    인구백명당부스터접종자수 = ('total_boosters_per_hundred', 'max')
).reset_index()

df_radar_vaccine = df_radar_vaccine[(df_radar_vaccine['iso_code'].\
isin(['OWID_ASI', 'OWID_EUR', 'OWID_OCE', 'OWID_NAM', 'OWID_SAM', 'OWID_AFR']))]

fig = go.Figure()

## scatterpolar 트레이스 추가
fig.add_trace(go.Scatterpolar(
    theta = df_radar_vaccine['location'],
    r = df_radar_vaccine['인구백명당백신접종완료율'],
    fill = 'toself'
    ))

## polar 속성 설정
fig.update_layout(polar =
    ## angularaxis 속성 설정
    dict(angularaxis =
    dict(ticktext = ('아프리카', '아시아', '유럽', '북미', '오세아니아', '남미'),
         tickvals = ('Africa', 'Asia', 'Europe', 'North America', 
                    'Oceania', 'South America'),
         linewidth = 2, linecolor = 'black', gridcolor = 'gray'),
    ## radialaxis 속성 설정
    radialaxis =
    dict(linewidth = 2, linecolor = 'dodgerblue', gridcolor = 'skyblue', 
        nticks = 5, ticksuffix = '%', title = '백신 접종률')),
    title = dict(text = '대륙별 백신 접종률', x = 0.5),
    margin = margins_P)

fig.show()

In [ ]:
# 덤벨차트

## 덤벨 차트를 위한 데이터 전처리
df_dumbbell = df_covid19_stat.copy().reset_index()
df_dumbbell = df_dumbbell.loc[(df_dumbbell['continent'].isna() == False) & 
                            (df_dumbbell['인구수'] > 10000000)].\
groupby('continent')['십만명당사망자수'].agg([('min', 'min'),
                                    ('max','max')]).reset_index()

fig = go.Figure()

## 최소값 트레이스 추가
fig.add_trace(go.Scatter(
    mode = 'markers+text', name = '최소',
    x = df_dumbbell['min'], y = df_dumbbell['continent'],
    text = round(df_dumbbell['min'], 1), textposition = 'bottom center',
    marker = dict(size = 8, color = 'skyblue')))

## 최대값 트레이스 추가
fig.add_trace(go.Scatter(
    mode = 'markers+text', name = '최대',
    x = df_dumbbell['max'], y = df_dumbbell['continent'],
    text = round(df_dumbbell['max'], 1), textposition = 'bottom center',
    marker = dict(size = 8, color = 'darkblue', symbol = 'circle-open')))

## 덤벨 차트용 세그먼트 추가
for index, row in df_dumbbell.iterrows():
    fig.add_shape(type="line", xref="x", yref="y",
    x0=row['min'], y0=row['continent'], x1=row['max'], y1=row['continent'],
    line=dict(color="gray",width=3))

fig.update_layout(title = dict(text = '대륙별 10만명당 사망자수 차이', x = 0.5),
    xaxis = dict(title = '10만명당 사망자수'),
    yaxis = dict(title = '', autorange = 'reversed'),
    margin = margins_P)

fig.show()

In [ ]:
# 파이 차트

df_pie = df_취업률_500.groupby('대계열').sum('졸업자수').reset_index()[['대계열', '졸업자수']]

fig = go.Figure()

## value와 labels를 매핑한 파이 trace 추가
fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', sort = True))
    
fig.update_layout(title = dict(text = '대학 계열별 졸업생 분포', x = 0.5))

fig.show()

In [ ]:
# 파이차트에 라벨표시

#####################################
fig = go.Figure()

## value를 매핑한 파이 trace 추가
fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', textinfo = 'value'))

fig.update_layout(title = dict(text = 'textinfo = value', x = 0.5))

fig.show()

#####################################
fig = go.Figure()

## value와 percent를 매핑한 파이 trace 추가
fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', textinfo = 'value+percent'))

fig.update_layout(title = dict(text = 'textinfo = value+percent', x = 0.5))

fig.show()

#####################################
fig = go.Figure()

## value와 label를 매핑한 파이 trace 추가
fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', textinfo = 'label+value'))

fig.update_layout(title = dict(text = 'textinfo = label+value', x = 0.5))

fig.show()

#####################################
fig = go.Figure()

## label과 percent를 매핑한 파이 trace 추가
fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', textinfo = 'label+percent'))

fig.update_layout(title = dict(text = 'textinfo = label+percent', x = 0.5))

fig.show()

In [ ]:
# 도넛차트

fig = go.Figure()

fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', sort = True,
    ## hole을 사용한 도넛 차트
    hole = 0.3))

fig.add_annotation(x = 0.5, y = 0.5, text = '<b>졸업생수</b>',
                showarrow = False, xanchor = 'center',
                font = dict(size = 20))

fig.update_layout(title = dict(text = '대학 계열별 졸업생 분포', x = 0.5))

fig.show()

In [ ]:
# 파이차트의 강조

fig = go.Figure()

fig.add_trace(go.Pie(
    values = df_pie['졸업자수'], labels = df_pie['대계열'],
    direction = 'clockwise', sort = True, hole = 0.3,
    ## 파이 차트 강조 설정
    pull = (0, 0.2, 0, 0, 0, 0, 0)))

fig.add_annotation(x = 0.5, y = 0.5, text = '<b>졸업생수</b>',
    showarrow = False, xanchor = 'center',
    font = dict(size = 20))

fig.update_layout(title = dict(text = '대학 계열별 졸업생 분포', x = 0.5))

fig.show()

In [ ]:
# 선버스트 차트

## 선버스트 차트를 위한 데이터 전처리
df_sunburst = df_취업률_500.groupby(['대계열', '중계열']).sum()['졸업자수'].reset_index()

all_sum = df_취업률_500.sum()['졸업자수']

계열_sum = df_취업률_500.groupby('대계열').sum().reset_index()['졸업자수'].values.tolist()

fig = go.Figure()

fig.add_trace(go.Sunburst(
    ## sunburst 트레이스의 labels 설정
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + 
    df_sunburst['중계열'].tolist(),
    ## sunburst 트레이스의 parents 설정
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(),
    ## sunburst 트레이스의 values 설정
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(),
    branchvalues = 'total'))

fig.show()

In [ ]:
# branchvalues = values로 설정된 아이템의 합계 산출 방법을 설정
# - total = values의 아이템이 모든 하위 항목값으로 간주. (50, 20, 20) -> (40%, 40%)
# - ramainder = 부모 섹터와 자식 섹터에 해당하는 values를 모두 더해서 비율을 산출. 20/(50+20+20) = 22%

## branchvalues = 'total' 선버스트 차트
fig = go.Figure()

fig.add_trace(go.Sunburst(
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + 
        df_sunburst['중계열'].tolist(),
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(),
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(),
    branchvalues = 'total'))

fig.show()

## branchvalues = 'remainder' 선버스트 차트
fig = go.Figure()

fig.add_trace(go.Sunburst(
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + 
    df_sunburst['중계열'].tolist(),
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(),
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(),
    branchvalues = 'remainder'))
    
fig.show()

In [ ]:
# insidetextorientation
# 트레이스에 표시되는 내부 정보 문자열이 표시되는 각도를 설정

## insidetextorientation = 'radial' 선버스트 차트
fig = go.Figure()

fig.add_trace(go.Sunburst(
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + df_sunburst['중계열'].tolist(),
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(),
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(),
    branchvalues = 'total', insidetextorientation = 'radial'))

fig.show()

## insidetextorientation = 'horizontal' 선버스트 차트
fig = go.Figure()

fig.add_trace(go.Sunburst(
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + df_sunburst['중계열'].tolist(),
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(),
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(),
    branchvalues = 'total', insidetextorientation = 'horizontal'))

fig.show()

In [ ]:
# 트리맵
# 선버스트의 사각형 버전. sunburst와 같이 labels, parents, values 속성을 동일하게 사용 가능

fig = go.Figure()

fig.add_trace(go.Treemap(
    ## treemap 트레이스의 labels 설정
    labels = ['전체'] + df_sunburst['대계열'].unique().tolist() + df_sunburst['중계열'].tolist(),
    ## treemap 트레이스의 parents 설정
    parents = [''] + ['전체'] * 7 + df_sunburst['대계열'].tolist(),
    ## treemap 트레이스의 values 설정
    values = [all_sum] + 계열_sum + df_sunburst['졸업자수'].values.tolist(),
    textinfo = 'label+value+percent parent+percent entry'))

fig.show()

In [ ]:
# CHAPTER 9 시간과 흐름의 시각화
# 선그래프

## 원본데이터 보존을 위해 데이터프레임 복사
total_deaths_5_nations_by_day = df_covid19.copy()

## 5개국 데이터로 전처리
total_deaths_5_nations_by_day = total_deaths_5_nations_by_day[
    (total_deaths_5_nations_by_day['iso_code'].isin(
        ['KOR', 'USA', 'JPN', 'GBR', 'FRA']))].dropna(
    subset = ['total_deaths_per_million'])

## 라인 타입 설정을 위한 딕셔너리 정의
nations = {'France':'0', 'Japan':'1', 'South Korea':'2', 
           'United Kingdom':'3', 'United States':'4'}

## Plotly 초기화
fig = go.Figure()

## 국가별로 트레이스를 추가
for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'], y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))

## layout의 제목, 축제목, 여백 속성 설정
fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
    xaxis = dict(title = ''),
    yaxis = dict(title = '10만명당 사망자수 누계'))
    
fig.show()

In [ ]:
# 주석을 사용한 선 그래프

fig = go.Figure()

## 국가별로 그룹화하여 for 루프 실행
for location, group in total_deaths_5_nations_by_day.groupby('location'):
    ## 각국의 백만명당 사망자수 scatter 트레이스 생성
    fig.add_trace(go.Scatter(mode = 'lines',
        x = group['date'],
        y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True, showlegend = False
    ))
    ## 각국의 마지막 날 옆에 국가명 scatter 트레이스 생성
    fig.add_trace(go.Scatter(
        mode = 'text',
        x = group.loc[group['date'] == group['date'].max(), 'date'],
        y = group.loc[group['date'] == group['date'].max(), 'total_deaths_per_million'],
        text = group.loc[group['date'] == group['date'].max(), 'location'],
        showlegend = False,
        textposition = 'middle right'
    ))

## 설날 표시를 위한 주석 추가
fig.add_annotation(
    x = '2022-02-01',
    y = total_deaths_5_nations_by_day.loc[
        (total_deaths_5_nations_by_day['date'] == '2022-02-01') & 
        (total_deaths_5_nations_by_day['iso_code'] == 'KOR'), 
        'total_deaths_per_million'].values[0],
    text = '설날', showarrow=True, arrowhead=1, arrowsize = 1.5
)

## layout의 제목, xaxis, yaxis 설정
fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
    xaxis = dict(title = '',
    range = [total_deaths_5_nations_by_day['date'].min(),
    total_deaths_5_nations_by_day['date'].max() +
    timedelta(days=180)]),
    yaxis = dict(title = '10만명당 사망자수 누계'))

fig.show()

In [ ]:
# rangeslider를 사용한 선 그래프

fig = go.Figure()

for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'], y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))

fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
    xaxis = dict(title = '',
    ## rangeslider 속성 설정
    rangeslider = dict(visible = True)),
    yaxis = dict(title = '10만명당 사망자수 누계'))

fig.show()

In [ ]:
# rangeselector를 사용한 선 그래프
# 명확한 기간 설정 가능

fig = go.Figure()

for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'],
        y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))

fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
    xaxis = dict(title = '',
    ## rangeslider 속성 설정
    rangeslider = dict(visible = True),
                 ## rangeselector 속성 설정
                 rangeselector=dict(
                     ## rangeselector의 buttons 속성 설정
                     buttons=list([
                         dict(count=7, label='1 Week before', 
                              step='day', stepmode="backward"),
                         dict(count=1, label='1 month before', 
                              step='month', stepmode="backward"),
                         dict(count=6, label='6 months before', 
                              step="month", stepmode="backward"),
                         dict(count=1, label='new years day', 
                              step="year", stepmode="todate"),
                         dict(count=1, label='1 year before', 
                              step="year", stepmode="backward")
    ]))),
    yaxis = dict(title = '10만명당 사망자수 누계'))

fig.show()

In [ ]:
# 호버 모드를 사용한 선 그래프

## 호버모드가 x인 시각화
fig = go.Figure()
for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'],
        y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))
fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
    xaxis = dict(title = ''),
    yaxis = dict(title = '10만명당 사망자수 누계'),
    ## 호버 모드 설정
    hovermode="x")
fig.show()

## 호버모드가 y인 시각화
fig = go.Figure()
for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'],
        y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))
fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
    xaxis = dict(title = ''),
    yaxis = dict(title = '10만명당 사망자수 누계'),
    ## 호버 모드 설정
    hovermode="y")
fig.show()

## 호버모드가 x unified인 시각화
fig = go.Figure()
for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'],
        y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))
fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
    xaxis = dict(title = ''),
    yaxis = dict(title = '10만명당 사망자수 누계'),
    ## 호버 모드 설정
    hovermode="x unified")
fig.show()

## 호버모드가 y unified인 시각화
fig = go.Figure()
for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'],
        y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))
fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
    xaxis = dict(title = ''),
    yaxis = dict(title = '10만명당 사망자수 누계'),
    ## 호버 모드 설정
    hovermode="y unified")
fig.show()

## 호버모드가 closet인 시각화
fig = go.Figure()
for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'],
        y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))
fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
    xaxis = dict(title = ''),
    yaxis = dict(title = '10만명당 사망자수 누계')
    ## 호버 모드 설정을 안하거나 'closet'값을 넣으면 됨
)
fig.show()


In [ ]:
# 스파이크 라인을 사용한 선 그래프
# across, toaxis, marker +로 합쳐서 사용 가능

fig = go.Figure()

for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'],
        y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))

fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
    xaxis = dict(title = '',
    ## X축의 spikemode 설정
    spikemode = 'across'),
    yaxis = dict(title = '10만명당 사망자수 누계',
    ## Y축의 spikemode 설정
    spikemode = 'toaxis'),
    hovermode="x")

fig.show()

In [ ]:
# 시간 축의 설정
# 눈금 라벨의 설정

fig = go.Figure()

for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'],
        y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))

fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),xaxis = dict(title = '', spikemode = 'across',
    ## X축 눈금 라벨 설정
    tickformat = '%Y년 %m월',
    ## X축 눈금 간격을 3달로 설정
    dtick = 'M3'),
    yaxis = dict(title = '10만명당 사망자수 누계',
    spikemode = 'toaxis'),
    hovermode="x")

fig.show()

In [ ]:
# 줌인/줌아웃에 반응하는 눈금의 설정

fig = go.Figure()

for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'],
        y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))

fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
    xaxis = dict(title = '',
                 spikemode = 'across', tickformat = '%Y년 %m월',
                 ## tickformatstops 설정, 밀리초단위
                 tickformatstops = (
                    ## 1000밀리초까지의 tickformat
                    dict(dtickrange=(None,1000), value="%H:%M:%S.%L 밀리초"),
                    ## 1초 ~ 1분까지의 tickformat
                    dict(dtickrange=(1000, 60000), value="%H:%M:%S 초"),
                    ## 1분 ~ 1시간까지의 tickformat
                    dict(dtickrange=(60000, 3600000), value="%H:%M 분"),
                    ## 1시간 ~ 1일까지의 tickformat
                    dict(dtickrange=(3600000, 86400000), value="%H:%M 시"),
                    ## 1일 ~ 1주까지의 tickformat
                    dict(dtickrange=(86400000, 604800000), value="%e. %b 일"),
                    ## 1주 ~ 1월까지의 tickformat
                    dict(dtickrange=(604800000, "M1"), value="%e. %b 주"),
                    ## 1월 ~ 1년까지의 tickformat
                    dict(dtickrange=("M1", "M12"), value="%b '%y 월"),
                    ## 1년 이상의 tickformat
                    dict(dtickrange=("M12",None), value="%Y 년")
    )),
    yaxis = dict(title = '10만명당 사망자수 누계',
    spikemode = 'toaxis'),
    hovermode="x")

fig.show()

In [ ]:
# financedatareader로 df만들기

import FinanceDataReader as fdr
from datetime import datetime, timedelta

start_day = datetime(2022, 10, 7)
end_day = start_day + timedelta(days = 100)

samsung_stock = fdr.DataReader('005930', start_day, end_day)
samsung_stock.head()

In [ ]:
# 캔들스틱 차트

fig = go.Figure()

## cadlestick 트레이스 추가
fig.add_trace(go.Candlestick(
    x = samsung_stock.index,
    ## OHLC 데이터 설정
    open = samsung_stock['Open'], close = samsung_stock['Close'],
    high = samsung_stock['High'], low = samsung_stock['Low']
    ))

fig.update_layout(title = dict(text = "삼성전자 Candlestick Chart", x = 0.5))

fig.show()

In [ ]:
# 캔들스틱 색 변경

fig = go.Figure()

fig.add_trace(go.Candlestick(
    x = samsung_stock.index,
    open = samsung_stock['Open'], close = samsung_stock['Close'],
    high = samsung_stock['High'], low = samsung_stock['Low'],
    ## 상승 시 선 색상 설정
    increasing = dict(line = dict(color = 'red')),
    ## 하락 시 선 색상 설정
    decreasing = dict(line = dict(color = 'blue'))))

fig.update_layout(title = dict(text = "삼성전자 Candlestick Chart", x = 0.5),
                ## rangeslider는 안 보이도록 설정
                xaxis = dict(rangeslider = dict(visible = False)))

fig.show()

In [ ]:
# 거래량 그래프 추가

## 서브플롯을 위한 라이브러리 로딩
from plotly.subplots import make_subplots

## 서브플롯 설정
fig = make_subplots(rows = 2, cols = 1, row_heights=[0.7, 0.3], shared_xaxes = True)

fig.add_trace(go.Candlestick(
    x = samsung_stock.index,
    open = samsung_stock['Open'], close = samsung_stock['Close'],
    high = samsung_stock['High'], low = samsung_stock['Low'],
    increasing = dict(line = dict(color = 'red')),
    decreasing = dict(line = dict(color = 'blue'))),
    row = 1, col = 1)

## 거래량 막대그래프인 bar 트레이스 추가
fig.add_trace(go.Bar(
    x = samsung_stock.index,
    y = samsung_stock['Volume'],
    marker = dict(color = 'gray')),
    row = 2, col =1)

## 서브플롯들의 Y축 제목 설정
fig.update_yaxes(title_text="주가", row=1, col=1)

fig.update_yaxes(title_text="거래량", row=2, col=1)

fig.update_layout(title = dict(text = "삼성전자 Candlestick Chart", x = 0.5),
                xaxis = dict(rangeslider = dict(visible = False)),
                showlegend = False)

fig.show()

In [ ]:
# 주말이 제거된 선 그래프
# 보간법 interpolation으로 해결

from plotly.subplots import make_subplots

fig = make_subplots(rows = 2, cols = 1, row_heights=[0.7, 0.3])

fig.add_trace(go.Candlestick(
    x = samsung_stock.index,
    open = samsung_stock['Open'], close = samsung_stock['Close'],
    high = samsung_stock['High'], low = samsung_stock['Low'],
    increasing = dict(line = dict(color = 'red')),
    decreasing = dict(line = dict(color = 'blue'))),
    row = 1, col = 1)

fig.add_trace(go.Bar(
    x = samsung_stock.index,
    y = samsung_stock['Volume'],
    marker = dict(color = 'gray'),
    xaxis = 'x2'),
    row = 2, col =1)

fig.update_xaxes(rangeslider = dict(visible = False),
    ## rangebreaks 설정
    rangebreaks = [
        ## 주말 제거
        dict(bounds=["sat", "mon"]),
        ## 특정 공휴일 제거
        dict(values=["2022-09-09", "2022-09-12", "2022-10-03", 
                     "2022-10-10", "2022-12-30"])],
    row = 2, col = 1)

fig.update_yaxes(title_text="거래량", row=2, col=1)

fig.update_layout(xaxis = dict(rangeslider = dict(visible = False),
    ## rangebreaks 설정
    rangebreaks = [
        ## 주말 제거
        dict(bounds=["sat", "mon"]),
        ## 특정 공휴일 제거
        dict(values=["2022-09-09", "2022-09-12", "2022-10-03",
                     "2022-10-10", "2022-12-30"]) ]),
    yaxis = dict(title_text="주가"),
    title = dict(text = "삼성전자 Candlestick Chart", x = 0.5),
    showlegend = False)

fig.show()

In [ ]:
# 주말이 있는 그래프와 없는 그래프

## 최근 100일간의 우리나라 코로나19 신규확진자 데이터 전처리
start_day = total_deaths_5_nations_by_day['date'].max() - timedelta(days=100)

total_deaths_5_nations_since_100day = total_deaths_5_nations_by_day.loc[(
    total_deaths_5_nations_by_day['iso_code'] =='KOR') & 
    (total_deaths_5_nations_by_day['date'] > start_day)]

## 서브플롯 설정
fig = make_subplots(rows = 2, cols = 1,
                    subplot_titles=('주말이 포함된 확진자수',
                                    '주말이 제거된 확진자수'))

## 주말이 포함된 scatter 트레이스 생성
fig.add_trace(go.Scatter(
    mode = 'lines',
    x = total_deaths_5_nations_since_100day['date'],
    y = total_deaths_5_nations_since_100day['new_cases'],
    line = dict(color = 'darkblue'),
    connectgaps = True),
    row = 1, col = 1)

## 주말이 제거된 scatter 트레이스 생성
fig.add_trace(go.Scatter(
    mode = 'lines',
    x = total_deaths_5_nations_since_100day['date'],
    y = total_deaths_5_nations_since_100day['new_cases'],
    line = dict(color = 'darkblue'),
    connectgaps = True),
    row = 2, col = 1)

fig.update_xaxes(rangebreaks=[
    dict(bounds=["sun", "tue"]),dict(values=['2022-03-02'])],
                 row = 2, col = 1)

fig.update_layout(title = dict(text = '우리나라의 코로나19 확진자수 추세', 
                               x = 0.5),
                  hovermode = "x unified", showlegend = False)

fig.show()

In [ ]:
# 이동평균 그리기

## 5일, 20일, 40일 이동평균 산출
samsung_stock['M5'] = samsung_stock['Close'].rolling(5).mean()
samsung_stock['M20'] = samsung_stock['Close'].rolling(20).mean()
samsung_stock['M40'] = samsung_stock['Close'].rolling(40).mean()
samsung_stock

from plotly.subplots import make_subplots

## 서브플롯 설정
fig = make_subplots(rows = 2, cols = 1, row_heights=[0.7, 0.3])

## candlestick 트레이스 생성
fig.add_trace(go.Candlestick(
    x = samsung_stock.index,
    open = samsung_stock['Open'], close = samsung_stock['Close'],
    high = samsung_stock['High'], low = samsung_stock['Low'],
    increasing = dict(line = dict(color = 'red')),
    decreasing = dict(line = dict(color = 'blue')), showlegend = False
    ), row = 1, col = 1)

## 5일 이동평균 scatter 트레이스 추가
fig.add_trace(go.Scatter(
        mode = 'lines',
        x = samsung_stock.index, y = samsung_stock['M5'],
        name = '5일 이동평균', line = dict(dash = 'solid')),
        row = 1, col = 1)

## 20일 이동평균 scatter 트레이스 추가
fig.add_trace(go.Scatter(
        mode = 'lines',
        x = samsung_stock.index, y = samsung_stock['M20'],
        name = '20일 이동평균', line = dict(dash = 'dash')),
        row = 1, col = 1)

## 40일 이동평균 scatter 트레이스 추가
fig.add_trace(go.Scatter(
        mode = 'lines',
        x = samsung_stock.index, y = samsung_stock['M40'],
        name = '40일 이동평균', line = dict(dash = 'dot')),
        row = 1, col = 1)

## 거래량 bar 트레이스 추가
fig.add_trace(go.Bar(
        x = samsung_stock.index, y = samsung_stock['Volume'],
        marker = dict(color = 'gray'), xaxis = 'x2', showlegend = False),
        row = 2, col =1)

fig.update_xaxes(rangeslider = dict(visible = False),
                 rangebreaks = [
                     dict(bounds=["sat", "mon"]),
                     dict(values=["2022-09-09", "2022-09-12", "2022-10-03", 
                                  "2022-10-10", "2022-12-30"]
                     )], 
                row = 2, col = 1)

fig.update_yaxes(title_text="거래량", row=2, col=1)

fig.update_layout(
    xaxis = dict(rangeslider = dict(visible = False),
                 rangebreaks = [
                     dict(bounds=["sat", "mon"]),
                     dict(values=["2022-09-09", "2022-09-12", 
                                  "2022-10-03", "2022-10-10", "2022-12-30"])
                     ]),
    yaxis = dict(title_text="주가"),
    title = dict(text = "삼성전자 Candlestick Chart", x = 0.5))

fig.show()

In [ ]:
# 폭포수 차트

## 전일 종가 대비 등락가 전처리
samsung_stock['lag'] = samsung_stock['Close'] - samsung_stock['Close'].shift(1)

fig = go.Figure()

## waterfall 트레이스 생성
fig.add_trace(go.Waterfall(
    name = "등락", orientation = "v",
    x = samsung_stock.index,
    y = samsung_stock['lag'],
    increasing = dict(marker = dict(color = 'red')),
    decreasing = dict(marker = dict(color = 'blue'))))

fig.update_layout(
    xaxis = dict(rangeslider = dict(visible = False),
                 rangebreaks = [dict(bounds=["sat", "mon"]),
                                dict(values=["2022-09-09", "2022-09-12", 
                                             "2022-10-03", "2022-10-10"])
                                ]),
                  yaxis = dict(title_text="주가 등락(원)"),
                  title = dict(text = "삼성전자 주가 Waterfall Chart", x = 0.5),
                  showlegend = False)

fig.show()

In [ ]:
# 퍼널 차트

## funnel 트레이스를 위한 데이터 복사
df_취업률_funnel = df_취업률.copy()

## funnel 트레이스를 위한 데이터 전처리
df_취업률_funnel['전체취업자'] = df_취업률_funnel[['취업자_교외취업자_계', '취업자_교내취업자_계']].sum(axis = 1)

df_취업률_funnel = df_취업률_funnel[['전체취업자', '1차 유지취업자_계', '2차 유지취업자_계', '3차 유지취업자_계', '4차 유지취업자_계']].sum().reset_index()

df_취업률_funnel.columns = ['구분', '취업자수']

fig = go.Figure()

## funnel 트레이스 생성
fig.add_trace(go.Funnel(
    x = df_취업률_funnel['취업자수'],
    y = df_취업률_funnel['구분'],
    text = df_취업률_funnel['취업자수'],
    textinfo = "text+percent initial"))

fig.update_layout(title = dict(text = '유지취업자 Funnel Chart', x = 0.5),
                  yaxis = dict(categoryorder = "total descending"))

fig.show()

In [ ]:
# 누적 퍼널 차트
# funnelmode의 기본값이 stack이기 때문에 두 개 이상의 funnel 트레이스가 자동으로 누적 퍼널 차트로 그려짐

## stack funnel 트레이스를 위한 데이터 복사
df_취업률_funnel_stack = df_취업률.copy()

## stack funnel 트레이스를 위한 데이터 전처리
df_취업률_funnel_stack['전체취업자'] = df_취업률_funnel_stack[['취업자_교외취업자_계',
'취업자_교내취업자_계']].sum(axis = 1)

df_취업률_funnel_stack = df_취업률_funnel_stack.groupby('과정구분')[['전체취업자',
'1차 유지취업자_계', '2차 유지취업자_계', '3차 유지취업자_계', '4차 유지취업자_계']].agg(
    전체취업자 = ('전체취업자', sum),
    유지취업자_1차 = ('1차 유지취업자_계', sum),
    유지취업자_2차 = ('2차 유지취업자_계', sum),
    유지취업자_3차 = ('3차 유지취업자_계', sum),
    유지취업자_4차 = ('4차 유지취업자_계', sum))

df_취업률_funnel_stack = df_취업률_funnel_stack.transpose()

fig = go.Figure()

## stack funnel 트레이스 생성
fig.add_trace(go.Funnel(
    x = df_취업률_funnel_stack['전문대학과정'],
    y = df_취업률_funnel_stack.index,
    name = '전문대학과정',
    text = df_취업률_funnel_stack['전문대학과정'],
    textinfo = "text+percent initial")
    )

fig.add_trace(go.Funnel(
    x = df_취업률_funnel_stack['대학과정'],
    y = df_취업률_funnel_stack.index,
    name = '대학과정',
    text = df_취업률_funnel_stack['대학과정'],
    textinfo = "text+percent initial")
    )

fig.update_layout(title = dict(text = '유지취업자 Funnel Chart', x = 0.5),
                  yaxis = dict(categoryorder = "total descending"))

fig.show()

In [ ]:
# 퍼널 영역 차트
# 퍼널차트는 x, y 축을 매핑하여 사용하지만 funnelarea 트레이는 text, values를 사용하여 값과 표시되는 라벨을 설정

## funnelarea 트레이스를 위한 데이터 전처리
df_funnelarea = df_covid19_100_wide[['확진자_아시아', '확진자_유럽', '확진자_북미', '확진자_남미', '확진자_아프리카', '확진자_오세아니아']].sum().reset_index()

df_funnelarea.columns = ('대륙', '전체확진자수')

fig = go.Figure()

## funnelarea 트레이스 생성
fig.add_trace(go.Funnelarea(
    text = df_funnelarea['대륙'],
    values = df_funnelarea['전체확진자수'],
    textinfo = "text+value+percent"))

fig.update_layout(title = dict(text = '최근 100일간 대륙별 확진자수 Funnelarea 차트', x = 0.5))

fig.show()

In [ ]:
# 생키 다이어그램

## 과정구분 데이터에 따른 코드값을 반환하는 함수 생성
def cat_1(row):
    key = row['과정구분']
    value = {"전문대학과정" : 0, "대학과정": 1, "대학원과정" : 2}.get(key)
    return value

## 졸업구분 데이터에 따른 코드값을 반환하는 함수 생성
def cat_2(row):
    key = row['졸업구분']
    value = {"취업" : 3, "진학": 4, "취업불가" : 5, "외국인" : 6, "제외인정" : 7, "기타" : 8, "미상" : 9}.get(key)
    return value

## sankey 트레이스를 위한 데이터 전처리
df_sankey = df_취업률.iloc[:, [2, 10, 19, 20, 21, 22, 23, 24]].groupby('과정구분').sum().reset_index()

df_sankey.columns = ('과정구분', '취업', '진학', '취업불가', '외국인', '제외인정', '기타', '미상')

## 데이터 형태를 변환
df_sankey = df_sankey.melt(id_vars = '과정구분', var_name = '졸업구분', value_name = '학생수').sort_values('과정구분')

## 과정구분_node와 졸업구분_node 값 설정
df_sankey['과정구분_node']= df_sankey.apply(cat_1, axis=1)
df_sankey['졸업구분_node']= df_sankey.apply(cat_2, axis=1)

In [ ]:
import numpy as np

## 왼쪽 노드로 사용할 변량을 from에 저장
node_from = df_sankey['과정구분'].unique()

## 오른쪽 노드로 사용할 변량을 to에 저장
node_to = df_sankey['졸업구분'].unique()

## 전체 노드 배열 생성
node = np.concatenate((node_from, node_to))

In [ ]:
fig = go.Figure()

fig.add_trace(go.Sankey(
    orientation = "h",
    node = dict(
        label = node,
        color = ['lightblue'] * 3 + ['darkblue'] * 7,
        pad = 15, thickness = 20,
        line = dict(color = "black", width = 0.5)),
    link = dict(
        source = df_sankey['과정구분_node'],
        target = df_sankey['졸업구분_node'],
        value = df_sankey['학생수'])))

fig.update_layout(title = dict(text = '대학과정별 졸업자의 졸업 후 진로', x = 0.5))

fig.show()

In [ ]:
# CHAPTER 10
# number indicator -> delta나 guage와 함께 전체 인디케이터를 구성하는 일부분으로 사용되는 것이 일반적이다.

from datetime import datetime, timedelta

## indicator 트레이스를 위한 데이터 전처리
today = total_deaths_5_nations_by_day['date'].max()

number_KOR = total_deaths_5_nations_by_day.loc[(
    total_deaths_5_nations_by_day['date']==total_deaths_5_nations_by_day['date'].max()) 
    & (total_deaths_5_nations_by_day['iso_code']=='KOR')]

number_KOR = number_KOR.loc[:,'total_deaths_per_million'].values[0]

fig = go.Figure()

## scatter 트레이스 생성
for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(
        mode = 'lines',
        x = group['date'], y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))
    
fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
                  xaxis = dict(title = ''),
                  yaxis = dict(title = '10만명당 사망자수 누계', domain = (0, 0.8)))

## number 모드의 indicator 트레이스 추가
fig.add_trace(go.Indicator(mode = 'number', value = number_KOR,
    title = dict(text = '<b>한국 코로나 사망자(10만명당)</b><br>' + 
                str(today.year) + '년' + str(today.month) + '월' + 
                str(today.day) + '일\n',
    font = dict(family = 'AppleGothic', size = 15)),
    ## number 속성 설정
    number = dict(font = dict(family = 'AppleGothic', size = 15),
                suffix = '명'),
    domain = dict(x = (0.4, 0.6), y = (0.8, 0.9))))

fig.show()

In [ ]:
# delta indicator

number1_KOR = total_deaths_5_nations_by_day.loc[(
    total_deaths_5_nations_by_day['date']==total_deaths_5_nations_by_day['date'].max() - timedelta(days = 1)) & 
    (total_deaths_5_nations_by_day['iso_code']=='KOR')]

number1_KOR = number1_KOR.loc[:,'total_deaths_per_million'].values[0]

fig = go.Figure()

for location, group in total_deaths_5_nations_by_day.groupby('location'):
    fig.add_trace(go.Scatter(mode = 'lines',
        x = group['date'], y = group['total_deaths_per_million'],
        line = dict(dash = nations[location]),
        name = location, connectgaps = True))

fig.update_layout(title = dict(text = '코로나19 사망자수 추세', x = 0.5),
                xaxis = dict(title = ''),
                yaxis = dict(title = '10만명당 사망자수 누계', domain = (0, 0.8)))

## number+delta 모드의 indicator 트레이스 추가
fig.add_trace(go.Indicator(
    mode = 'number+delta', value = number_KOR,
    title = dict(text = '<b>한국 코로나 사망자(10만명당)</b><br>' + str(today.year) + '년' + str(today.month) + '월' + str(today.day) + '일\n',
                font = dict(family = 'AppleGothic', size = 15)),
    number = dict(font = dict(family = 'AppleGothic', size = 15), suffix = '명'),
    ## delta 속성 설정
    delta = dict(reference = number1_KOR, position = 'right',
                increasing = dict(color = 'red'),
                decreasing = dict(color = 'blue'),
                font = dict(family = 'AppleGothic', size = 10)),
    domain = dict(x = (0.4, 0.6), y = (0.8, 0.9))))

fig.show()

In [ ]:
# guage indicator

## 게이지 indicator 트레이스를 위한 데이터 전처리
deaths_per_million_in_lateast = total_deaths_5_nations_by_day[
    total_deaths_5_nations_by_day['new_deaths_per_million'].isna() == False]

deaths_per_million_in_lateast = pd.merge(
    deaths_per_million_in_lateast.groupby('location')['date'].max(), 
    deaths_per_million_in_lateast, on = ("location", 'date'))\
[['iso_code', 'location', 'date', 'new_deaths_per_million']]

df_gauge = pd.merge(
    deaths_per_million_in_lateast, 
    total_deaths_5_nations_by_day.groupby('location')['new_deaths_per_million'].\
    max().reset_index(), on = 'location').sort_values('location')

df_gauge.columns = ('iso_code', 'location', 'date', '최근사망자', '최대사망자')

fig = go.Figure()

## 한국 게이지 인디케이터 생성
fig.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[2, 1],
    domain = dict(row = 1, column = 1), value = df_gauge.iloc[2, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[2, 4]*1.2)),
                 steps = [
                     dict(range = (0, (df_gauge.iloc[2, 4])*1.2*0.5), 
                          color = "lightgray"),
                     dict(range = ((df_gauge.iloc[2, 4])*1.2*0.5, 
                                   (df_gauge.iloc[2, 4])*1.2*0.75),
                          color = "darkgray"),
                     dict(range = ((df_gauge.iloc[2, 4])*1.2*0.75, 
                                   (df_gauge.iloc[2, 4])*1.2),
                          color = "gray")],
                 threshold = dict(line = dict(color = 'white'),
                                  value = df_gauge.iloc[2, 4]),
                 bar = dict(color = "darkblue")),
    number = dict(suffix = '명')))

## 프랑스 게이지 인디케이터 생성
fig.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[0, 1],
    domain = dict(row = 0, column = 0), value = df_gauge.iloc[0, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[0, 4]*1.2)),
                 steps = [
                     dict(range = (0, (df_gauge.iloc[0, 4])*1.2*0.5), 
                          color = "lightgray"),
                     dict(range = ((df_gauge.iloc[0, 4])*1.2*0.5, 
                                   (df_gauge.iloc[0, 4])*1.2*0.75),
                          color = "darkgray"),
                     dict(range = ((df_gauge.iloc[0, 4])*1.2*0.75, 
                                   (df_gauge.iloc[0, 4])*1.2),
                          color = "gray")],
                 threshold = dict(line = dict(color = 'white'),
                                  value = df_gauge.iloc[0, 4]),
                 bar = dict(color = "darkblue")),
    number = dict(suffix = '명')))

## 일본 게이지 인디케이터 생성
fig.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[1, 1],
    domain = dict(row = 0, column = 2), value = df_gauge.iloc[1, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[1, 4]*1.2)),
                 steps = [
                     dict(range = (0, (df_gauge.iloc[1, 4])*1.2*0.5), 
                          color = "lightgray"),
                     dict(range = ((df_gauge.iloc[1, 4])*1.2*0.5, 
                                   (df_gauge.iloc[1, 4])*1.2*0.75),
                          color = "darkgray"),
                     dict(range = ((df_gauge.iloc[1, 4])*1.2*0.75, 
                                   (df_gauge.iloc[1, 4])*1.2),
                          color = "gray")],
                 threshold = dict(line = dict(color = 'white'),
                                  value = df_gauge.iloc[1, 4]),
                 bar = dict(color = "darkblue")),
    number = dict(suffix = '명')))

## 영국 게이지 인디케이터 생성
fig.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[3, 1],
    domain = dict(row = 2, column = 0), value = df_gauge.iloc[3, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[3, 4]*1.2)),
                 steps = [
                     dict(range = (0, (df_gauge.iloc[3, 4])*1.2*0.5), 
                          color = "lightgray"),
                     dict(range = ((df_gauge.iloc[3, 4])*1.2*0.5, 
                                   (df_gauge.iloc[3, 4])*1.2*0.75),
                          color = "darkgray"),
                     dict(range = ((df_gauge.iloc[3, 4])*1.2*0.75, 
                                   (df_gauge.iloc[3, 4])*1.2),
                          color = "gray")],
                 threshold = dict(line = dict(color = 'white'),
                                  value = df_gauge.iloc[3, 4]),
                 bar = dict(color = "darkblue")),
    number = dict(suffix = '명')))

## 미국 게이지 인디케이터 생성
fig.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[4, 1],
    domain = dict(row = 2, column = 2), value = df_gauge.iloc[4, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[4, 4]*1.2)),
                 steps = [
                     dict(range = (0, (df_gauge.iloc[4, 4])*1.2*0.5), 
                          color = "lightgray"),
                     dict(range = ((df_gauge.iloc[4, 4])*1.2*0.5, 
                                   (df_gauge.iloc[4, 4])*1.2*0.75),
                          color = "darkgray"),
                     dict(range = ((df_gauge.iloc[4, 4])*1.2*0.75, 
                                   (df_gauge.iloc[4, 4])*1.2),
                          color = "gray")],
                 threshold = dict(line = dict(color = 'white'),
                                  value = df_gauge.iloc[4, 4]),
                 bar = dict(color = "darkblue")),
    number = dict(suffix = '명')))

fig.update_layout(grid=dict(rows=3, columns=3),
                  title = dict(text = '10만명당 사망자수(최근 공식발표 기준)', 
                               x = 0.5))

fig.show()

In [ ]:
# 불릿 인디케이터
# step: 눈금 단계, value: 데이터양의 막대, threshold: 임계치
# 배열형태로 단계값들을 지정, 표시할 수치의 현재 값, 수치의 목표치나 안정적 데이터 구간을 표시하는 데 사용

fig = go.Figure()

## 한국 불릿 인디케이터 생성
fig.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[2, 1],
    domain = dict(x = (0.3,0.8), y = (0.82, 0.9)), value = df_gauge.iloc[2, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[2, 4]*1.2)), shape = "bullet",
                 steps = [
                     dict(range = (0, (df_gauge.iloc[2, 4])*1.2*0.5), 
                          color = "lightgray"),
                     dict(range = ((df_gauge.iloc[2, 4])*1.2*0.5, 
                                   (df_gauge.iloc[2, 4])*1.2*0.75),
                          color = "darkgray"),
                     dict(range = ((df_gauge.iloc[2, 4])*1.2*0.75, 
                                   (df_gauge.iloc[2, 4])*1.2),
                          color = "gray")],
                 threshold = dict(line = dict(color = 'white'), 
                                  value = df_gauge.iloc[2, 4]),
                 bar = dict(color = "darkblue")), number = dict(suffix = '명')))

## 프랑스 불릿 인디케이터 생성
fig.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[0, 1],
    domain = dict(x = (0.3,0.8), y = (0.62, 0.7)), value = df_gauge.iloc[0, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[0, 4]*1.2)), shape = "bullet", 
                 steps = [
                     dict(range = (0, (df_gauge.iloc[0, 4])*1.2*0.5), 
                          color = "lightgray"),
                     dict(range = ((df_gauge.iloc[0, 4])*1.2*0.5, 
                                   (df_gauge.iloc[0, 4])*1.2*0.75),
                          color = "darkgray"),
                     dict(range = ((df_gauge.iloc[0, 4])*1.2*0.75, 
                                   (df_gauge.iloc[0, 4])*1.2),
                          color = "gray")],
                 threshold = dict(
                     line = dict(color = 'white'), value = df_gauge.iloc[0, 4]),
                 bar = dict(color = "darkblue")), number = dict(suffix = '명')))

## 일본 불릿 인디케이터 생성
fig.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[1, 1],
    domain = dict(x = (0.3,0.8), y = (0.42, 0.5)), value = df_gauge.iloc[1, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[1, 4]*1.2)), shape = "bullet",
                 steps = [
                     dict(range = (0, (df_gauge.iloc[1, 4])*1.2*0.5), 
                          color = "lightgray"),
                     dict(range = ((df_gauge.iloc[1, 4])*1.2*0.5, 
                                   (df_gauge.iloc[1, 4])*1.2*0.75),
                          color = "darkgray"),
                     dict(range = ((df_gauge.iloc[1, 4])*1.2*0.75, 
                                   (df_gauge.iloc[1, 4])*1.2),
                          color = "gray")],
                 threshold = dict(line = dict(color = 'white'), 
                                  value = df_gauge.iloc[1, 4]),
                 bar = dict(color = "darkblue")), number = dict(suffix = '명')))

## 영국 불릿 인디케이터 생성
fig.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[3, 1],
    domain = dict(x = (0.3,0.8), y = (0.22, 0.3)), value = df_gauge.iloc[3, 3],
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[3, 4]*1.2)),
                 shape = "bullet", steps = [
                     dict(range = (0, (df_gauge.iloc[3, 4])*1.2*0.5), 
                          color = "lightgray"),
                     dict(range = ((df_gauge.iloc[3, 4])*1.2*0.5, 
                                   (df_gauge.iloc[3, 4])*1.2*0.75),
                          color = "darkgray"),
                     dict(range = ((df_gauge.iloc[3, 4])*1.2*0.75, 
                                   (df_gauge.iloc[3, 4])*1.2),
                          color = "gray")],
                 threshold = dict(
                     line = dict(color = 'white'), value = df_gauge.iloc[3, 4]),
                 bar = dict(color = "darkblue")), number = dict(suffix = '명')))

## 미국 불릿 인디케이터 생성
fig.add_trace(go.Indicator(
    type = 'indicator', mode = "gauge+number", title = df_gauge.iloc[4, 1],
    domain = dict(x = (0.3,0.8), y = (0.02, 0.1)),
    value = df_gauge.iloc[4, 3], 
    gauge = dict(axis = dict(
        range = (0, df_gauge.iloc[4, 4]*1.2)),
                 shape = "bullet", steps = [
                     dict(range = (0, (df_gauge.iloc[4, 4])*1.2*0.5), 
                          color = "lightgray"),
                     dict(range = ((df_gauge.iloc[4, 4])*1.2*0.5, 
                                   (df_gauge.iloc[4, 4])*1.2*0.75),
                          color = "darkgray"),
                     dict(range = ((df_gauge.iloc[4, 4])*1.2*0.75, 
                                   (df_gauge.iloc[4, 4])*1.2),
                          color = "gray")],
                 threshold = dict(
                     line = dict(color = 'white'), value = df_gauge.iloc[4, 4]),
                 bar = dict(color = "darkblue")), number = dict(suffix = '명')))

fig.update_layout(grid=dict(rows=3, columns=3),
                  title = dict(text = '10만명당 사망자수(최근 공식발표 기준)', 
                               x = 0.5))
                               
fig.show()

In [ ]:
# 지도의 시각화
# scattergeo trace - 물리적 지도

## scattergeo 트레이스 생성
fig = go.Figure(go.Scattergeo())

## geo 속성 설정
fig.update_layout(geo = dict(
  resolution=50,
  showcoastlines=True, coastlinecolor='RebeccaPurple',
  showland=True, landcolor='LightGray',
  showocean=True, oceancolor='LightBlue',
  showlakes=True, lakecolor='Blue',
  showrivers=True, rivercolor='Blue'),
  height=300, margin={'r':0,'t':0,'l':0,'b':0})

fig.show()

In [ ]:
## scattergeo 트레이스 생성
fig = go.Figure(go.Scattergeo())

## geo 속성 설정
fig.update_layout(geo = dict(
  resolution=50,
  showcountries=True, countrycolor='darkBlue'),
  height=300, margin={'r':0,'t':0,'l':0,'b':0})

fig.show()

In [ ]:
# choropleth trace

## json 파일을 읽기 위한 라이브러리 로딩
import json

## 지역값 매핑을 위한 함수 정의
def cat(row):
    key = row['지역']
    value = {'강원' : '42','경기' : '41','경남' : '48','경북' : '47','광주' : '29','대구' :
    '27','대전' : '30','부산' : '26','서울' : '11','세종' : '36','울산' : '31','인천' : '28',
    '전남' : '46','전북' : '45','제주' : '50','충남' : '44','충북' : '43'}.get(key)
    return value

## json 파일의 데이터를 불러들임
geometry = json.load(open('TL_SCCO_CTPRVN.json', encoding='utf-8'))

## 충원율 데이터 불러들임
df_충원율 = pd.read_excel("고등 주요 01-시도별 신입생 충원율(2010-2022)_220825y.xlsx",
                       sheet_name = 'Sheet1', skiprows=(6), header = 0)

df_충원율 = df_충원율.iloc[:, 0:5]
df_충원율.columns = ('연도', '지역', '정원내모집인원', '정원내입학생수', '신입생충원율')
df_충원율['지역코드']= df_충원율.apply(cat, axis=1)
df_충원율 = df_충원율[(df_충원율['지역'] != '전국')&(df_충원율['연도'] == 2022)]

fig = go.Figure()

## chropleth 트레이스 생성
fig.add_trace(go.Choropleth(
    geojson=geometry, 
    featureidkey='properties.CTPRVN_CD',
    locations = df_충원율['지역코드'],
    z = df_충원율['신입생충원율'],
    colorscale="Blues",
    text = df_충원율['신입생충원율'],
    marker=dict(line=dict(width=1),
                opacity = 0.5),
    colorbar = dict(y = 0.5, yanchor = 'middle')))

fig.update_layout(
    title = dict(text = '22년 전국 대학 신입생 충원율', x = 0.5),
    ## geo 속성 설정
    geo=dict(
        showframe = True, fitbounds = "locations",
        visible = False, center=dict(lon=126.98, lat=37.56)),
    autosize=False,
    margin = dict(t = 50, b = 25, l = 25, r = 25, pad=4, autoexpand=True))

fig.show()

In [ ]:
# scattermapbox trace

# mapbox access key를 만들고 활용

## 대학 위경도 데이터 불러들임
df_univ = pd.read_excel("university.xlsx")

## scattermapbox 트레이스 추가
fig = go.Figure(go.Scattermapbox(
    lat=df_univ['lat'], lon=df_univ['lon'],
    mode='markers+text',
    marker=dict(symbol ='marker', size=15, color='blue'),
    text=df_univ['학교명'], textposition = 'top center'))

fig.update_layout(title = dict(text = '서울지역 주요 대학', x = 0.5),
                  autosize=True, hovermode='closest',
                  ## mapbox 속성 설정
                  mapbox=dict(
                    accesstoken= 'pk.eyJ1IjoiMnN0bmRhcmQiLCJhIjoiY2xicnZjczloMDEzajNvbnk2d255ZHp6ayJ9.A35ianZRE4hWac0ekBDhKg',
                    bearing=0, center=dict(lon=126.98, lat=37.56),
                    pitch=0, zoom=10, style="light"))

fig.show()